# CH.4 Classes & OOP

# Object Comparisons: “is” vs “==”

`==` 用於判斷內容有沒有一樣  
`is` 用來判斷兩個變數是否指向相同的 object (改一個，令一個也會跟著變動)

In [1]:
a = [1, 2, 3]
b = a

print(a == b)
print(a is b)

True
True


In [2]:
b[0] = 123
print(a)

[123, 2, 3]


如果建立令一個變數 c 為 a 的拷貝，他們不會指向相同 object

In [3]:
c = list(a)

print(a == c)
print(a is c)

True
False


# String Conversion (Every Class Needs a `__repr__` )
如果正常的把 class 給印出來(轉成字串或是在 interpreter session)  
只會看到他的名稱和記憶體位置

In [4]:
class Car:
  def __init__(self, color, mileage):
    self.color = color
    self.mileage = mileage

In [5]:
myCar = Car('red', 123)

print(myCar) #　轉成字串

myCar # interpreter session

但如果加上 `__str__` method 後，就能改變 print 出來的資訊  
`__str__` 是一種 python 的 dunder(doublke-underscore) methods  
他在 object 準備被轉成 str 時呼叫

In [6]:
class Car:
  def __init__(self, color, mileage):
    self.color = color
    self.mileage = mileage
  def __str__(self):
    return f'a {self.color} car'

In [7]:
myCar = Car('red', 123)

print(myCar)

myCar # 注意這個輸出

a red car


## \_\_str\_\_ vs \_\_repr\_\_
在上一個範例，在 interpreter session 檢查 myCar 時，依舊會輸出記憶體位置和名稱  
這是因為他們使用了兩種不同的 dunder `__str__` 和 `__repr__`



In [8]:
class Car:
  def __init__(self, color, mileage):
    self.color = color
    self.mileage = mileage
  
  def __repr__(self):
    return '__repr__ for Car'
  
  def __str__(self):
    return '__str__ for Car'

In [9]:
my_car = Car('red', 37281)

print(my_car)
my_car

__str__ for Car


__repr__ for Car

為了避免搞混，在需要檢查 str 或是 repr 時建議用對應的 function

In [10]:
print(str(my_car))
print(repr(my_car))

__str__ for Car
__repr__ for Car


一個實際的使用情況是，我們希望能直接 print 出好讀的時間表示

In [11]:
import datetime
today = datetime.date.today()

str(today)

'2022-08-05'

**repr** 主要是在幫助我們開發或 debug，因此會有關於資料型態的資訊

In [12]:
repr(today)

'datetime.date(2022, 8, 5)'

## Why Every Class Needs a \_\_repr\_\_

如果沒加上 `__str__` method，python 會回去找 `__repr__` method  
因此，建議都要幫 class 加上 `__repr__` method

In [13]:
class Car:
  def __init__(self, color, mileage):
    self.color = color
    self.mileage = mileage
  
  def __repr__(self):
    return (f'{self.__class__.__name__}('
            f'{self.color!r}, {self.mileage!r})')

In [14]:
my_car = Car('blue', 123)

repr(my_car)

"Car('blue', 123)"

In [15]:
str(my_car)

"Car('blue', 123)"

**Key Takeaways**
+ You can control to-string conversion in your own classes using
the __str__ and __repr__ “dunder” methods.
+ The result of __str__ should be readable. The result of
__repr__ should be unambiguous.
+ Always add a __repr__ to your classes. The default implementation for __str__ just calls __repr__.
+ Use __unicode__ instead of __str__ in Python 2.

## 3.4 Defining Your Own Exception Classes
使用內建的 exception 似乎很合理，但在維護上往往讓人不知道明確的錯誤點在哪

```python
def validate(name):
  if len(name) < 10:
    raise ValueError

>>> validate('joe')
Traceback (most recent call last):
  File "<input>", line 1, in <module>
    validate('joe')
  File "<input>", line 3, in validate
    raise ValueError
ValueError
```
我們可以自訂義 exception 來讓程式更好維護

In [16]:
class NameTooShortError(ValueError):
  pass
  
def validate(name):
  if len(name) < 10:
    raise NameTooShortError(name)

In [17]:
# 這段將會出錯
# validate('tom') 

理解用法後，進一步延伸建立出自己的 grouped exception  
所有相關的 exception 都能繼承他  

In [18]:
# base
class BaseValidationError(ValueError):
  pass

# 延伸
class NameTooShortError(BaseValidationError):
  pass
class NameTooLongError(BaseValidationError):
  pass
class NameTooCuteError(BaseValidationError):
  pass  

如使用者不想明確定義是哪個 exception  
就能使用 base exception 替代

In [19]:
name = 'tom'

def handle_validation_error(BaseValidationError):
  pass

try:
  validate(name)
except BaseValidationError as err:
  handle_validation_error(err)

**Key Takeaways**
+ Defining your own exception types will state your code’s intent
more clearly and make it easier to debug.
+ Derive your custom exceptions from Python’s built-in
Exception class or from more specific exception classes
like ValueError or KeyError.
+ You can use inheritance to define logically grouped exception
hierarchies.

# 4.4 Cloning Objects for Fun and Profit
Assignment statements 在 python 中沒有建立 copy object，而是將變數名稱綁到 object 上(類似指標)  
這章節將介紹如何真的的拷貝 object ，我們先來看 python 內建的拷貝方式

```python
new_list = list(original_list)
new_dict = dict(original_dict)
new_set = set(original_set)
```
可惜的是他們只是 **shallow copies** 而已  
+ A shallow copy means constructing a new collection object and then
populating it with references to the child objects found in the original.
In essence, a shallow copy is only **one level deep**. The copying process
does not recurse and therefore **won’t create copies of the child objects
themselves.**  

意思是他們只拷貝**第一層**，child object 則不拷貝


## Making Shallow Copies
以下的例子 ys 是 xs 的 shallow copy，看起來彼此互相不影響

In [20]:
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
ys = list(xs) # Make a shallow copy

In [21]:
xs.append([123])

print(xs)
print(ys)

[[1, 2, 3], [4, 5, 6], [7, 8, 9], [123]]
[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


但進一步觀察 child object 時會發現，child object 還是共享的

In [22]:
xs[0][0] = 'xxx'

print(xs)
print(ys)

[['xxx', 2, 3], [4, 5, 6], [7, 8, 9], [123]]
[['xxx', 2, 3], [4, 5, 6], [7, 8, 9]]


另一種建立 shallow copy 的用法是 copy.copy()，或是 object.copy()

In [23]:
import copy

xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
ys = copy.copy(xs) # or xs.copy()

In [24]:
xs[0][0] = 'xxx'

print(xs)
print(ys)

[['xxx', 2, 3], [4, 5, 6], [7, 8, 9]]
[['xxx', 2, 3], [4, 5, 6], [7, 8, 9]]


## Making Deep Copies
deep copy 以後彼此互不影響  

In [25]:
import copy
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
zs = copy.deepcopy(xs)

In [26]:
xs[0][0] = 50
print(zs)

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]


## Copying Arbitrary Objects
建立 class 來觀察 shallow copy 和 deep copy 的差異  
Rectangle class 使用到了 Point class 作為成員  
在 shallow copy 後，child member 依舊相連


In [27]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y
  def __repr__(self):
    return f'Point({self.x!r}, {self.y!r})'

In [28]:
class Rectangle:
  def __init__(self, topleft, bottomright):
    self.topleft = topleft
    self.bottomright = bottomright
  def __repr__(self):
    return (f'Rectangle({self.topleft!r}, '
        f'{self.bottomright!r})')

In [29]:
import copy

a = Rectangle(Point(0, 0), Point(10, 10))
b = copy.copy(a)

print(a)
print(b)

Rectangle(Point(0, 0), Point(10, 10))
Rectangle(Point(0, 0), Point(10, 10))


In [30]:
a.topleft.x = 200

print(a)
# Point 底下的 x 對 Rectangle 來說是 child member 
# 他們依舊相連
print(b) 

Rectangle(Point(200, 0), Point(10, 10))
Rectangle(Point(200, 0), Point(10, 10))


如果只更改第一層的成員(topleft 和 bottomright)則不影響

In [31]:
a.topleft = Point(80, 80)

print(a)
print(b)

Rectangle(Point(80, 80), Point(10, 10))
Rectangle(Point(200, 0), Point(10, 10))


補充:  
在 python 官方文檔 [Shallow and deep copy operations](https://docs.python.org/3/library/copy.html) 有更詳細的說明，包含如何透過 ` __copy__()`, `__deepcopy__()` 定義 copy 行為

**Key Takeaways**
+ Making a shallow copy of an object won’t clone child objects.
Therefore, the copy is not fully independent of the original.
+ A deep copy of an object will recursively clone child objects. The
clone is fully independent of the original, but creating a deep
copy is slower.
+ You can copy arbitrary objects (including custom classes) with
the copy module.

# 4.5 Abstract Base Classes Keep Inheritance in Check
Abstract Base Classes (ABCs) 確保繼承他的 class 需要實作特定的 methods  
這章節將講述如何透過 python 內建的 abc module 來定義他們  
  
ABCs 應有以下特性:
+ 他們不能被實體化 (instantiating)
+ sub-class 忘了實做某些 method 會出現警告


In [32]:
class Base:
  def foo(self):
    raise NotImplementedError()
  def bar(self):
    raise NotImplementedError()

class Concrete(Base):
  def foo(self):
    return 'foo() called'
# Oh no, we forgot to override bar()...
# def bar(self):
# return "bar() called"

建立 base class 物件不會出現錯誤，但在使用 method 時就會

In [33]:
b = Base()
# NotImplementedError:
# b.foo() 

有方法沒實作，一呼叫就會出現錯誤

In [34]:
c = Concrete()
# NotImplementedError
# c.bar()

這跟最前面提到的似乎有些不同(還是能實體化)  
但我們能使用 abc module 來完善他們

In [35]:
from abc import ABCMeta, abstractmethod

class Base(metaclass=ABCMeta):
  
  @abstractmethod
  def foo(self):
    pass
  @abstractmethod
  def bar(self):
    pass

class Concrete(Base):
  def foo(self):
    pass

# We forget to declare bar() again...

指定 metaclass=ABCMeta 後，想要實例化 baseclass 會出現錯誤 (不信你試試)  
subclass 如果少實作 method 也不行


In [36]:
# c = Base()
# >>> TypeError: Can't instantiate abstract class Base with abstract methods bar, foo

In [37]:
# c = Concrete()
# TypeError: Can't instantiate abstract class Concrete with abstract methods bar

In [38]:
# Concreate 仍然是 subclass
assert issubclass(Concrete, Base)

**Key Takeaways**
+ Abstract Base Classes (ABCs) ensure that derived classes implement particular methods from the base class at instantiation
time.
+ Using ABCs can help avoid bugs and make class hierarchies easier to maintain.

# 4.6 What Namedtuples Are Good For
namedtuple 在 python 內是一個很特別的資料結構，可以作為 class 的替代方案  
在理解他以前先來看看內建的 tuple 型態，跟 list 很相似，但具有 immutable (不可更動特性)，一被建立就無法更動
  
還有兩個缺點:
+ 只能透過 int index 存取，這會影響可讀性
+ tuple 是 ad-hoc structure (臨時使用的資料結構)，很難保證兩個 tuple 間有相同數量的欄位和資料型態

```python
tup = ('hello', object(), 42)
>>> a = tup[0]
>>> tup[0] = 20
TypeError:
"'tuple' object does not support item assignment"
```
關於 [ad-hoc](https://stackoverflow.com/questions/1786735/what-does-ad-hoc-mean-in-programming) 在程式的意義  
some quick and dirty code without the intention of reuse

## Namedtuples to the Rescue
就是為了解決以上問題而但誕生的 
+ Namedtuples 遵守 write once, read many 原則，不能修改內容  
+ 可透過 unique (human-readable)
identifier 來存取元素  
  
Nametuple 使用範例:  
+ 第一個參數 SuperCar 是 typename，在 `__repr__` 呼叫時會顯示  
+ 第二個參數可為空白or逗號分隔的字串或是 list，代表這個 namedtuple 的欄位

In [39]:
from collections import namedtuple
Car = namedtuple('SuperCar' , 'color mileage')
# Car = namedtuple('SuperCar' , ['color', 'mileage'])

現在 nametuple Car 有兩個元素了，可透過名稱或是 index 來存取

In [40]:
# SuperCar will show when calling __repr__ method
my_car

Car('blue', 123)

In [41]:
my_car = Car('red', 300)

print(my_car.color)
print(my_car.mileage)

print(my_car[0])
print(my_car[1])

red
300
red
300


tuple unpacking 也是可行的

In [42]:
print(*my_car)

red 300


## Subclassing Namedtuples
你也可延伸 nametuple 的功能，幫其加上 method

In [43]:
from collections import namedtuple
Car = namedtuple('Car', ['color', 'mileage'])

In [44]:
class MyCarWithMethods(Car):
  def hexcolor(self):
    if self.color == 'red':
      return "#ff0000"
    else:
      return '#000000'

In [45]:
c = MyCarWithMethods('red', 1324)
c.hexcolor()

'#ff0000'

幫 nametuple 加上新的欄位是很麻煩，因為它不可更改  
更好的方式為創建一個新的 nametuple，並沿用舊的 nametuple 欄位  
`._fields` 可以取得欄位名稱 (複習一下，前單底線代表私有成員)

In [46]:
Car = namedtuple('Car', 'color mileage')
ElectricCar = namedtuple('ElectricCar', Car._fields + ('charge',))

In [47]:
ElectricCar('red', 1234, 45.0)

ElectricCar(color='red', mileage=1234, charge=45.0)

## Built-in Helper Methods
除了 `_fields` attribute，以下將介紹其他好用的 helper **methods**，他們都是前單底線開頭(私有成員)
  
`_asdict()` 回傳 nametuple 的內容為 dictionary



In [48]:
my_car._asdict()

OrderedDict([('color', 'red'), ('mileage', 300)])

In [49]:
import json
json.dumps(my_car._asdict())

'{"color": "red", "mileage": 300}'

另一個好用的 help 是 `_replace()`，建立了一個 shallow copy，並能更改欄位內的內容

In [50]:
my_car

SuperCar(color='red', mileage=300)

In [51]:
my_car._replace(color='blue')

SuperCar(color='blue', mileage=300)

## When to Use Namedtuples
nametuple 能增加程式碼的可讀性，那有什麼方能使用呢?  
在不希望資料更改(immutable)，又想有 dictionary 那樣的可讀性時候就能使用


**Key Takeaways**
+ collection.namedtuple is a **memory-efficient** shortcut to
manually define an **immutable** class in Python.
+ Namedtuples can help clean up your code by enforcing an
easier-to-understand structure on your data
+  Namedtuples provide a few **useful helper methods** that all start with a single underscore, but are part of the public interface.It’s okay to use them

# 4.7 Class vs Instance Variable Pitfalls
python 內有兩種 data attribute:
+ class variable
+ instances variable

**Class variables** 在 class 內定義及儲存在 class 內，不屬於任何一個 instance，被所有的 instance 共享。
+ 從 class 層面去更改會影響所有 instance
+ 從 instance 層面去更改則會建立一個同名的 instance variable，呼叫變數時就不會去存取原本的 class variable 
  
**Instance variables** 永遠跟特定的 object instance 綁定，不同 instance 不共享變數



In [52]:
class Dog:
  num_legs = 4 # <- Class variable
  def __init__(self, name):
    self.name = name # <- Instance variable

In [53]:
jack = Dog('jack')
tom = Dog('time')

jack.name, tom.name

('jack', 'time')

如果要存取 class variable，除了從 instance 存取之外，也可以直接從 class 存取

In [54]:
Dog.num_legs

4

但如果透過 class 存取 **instance variable**，那將會出錯

```python
>>> Dog.name
AttributeError:
"type object 'Dog' has no attribute 'name'"
```

前面提到 class variable 是共享的  
從 class 層面更改 class variable 將會**影響到所有 instance**

In [55]:
Dog.num_legs = 100

jack.num_legs, tom.num_legs

(100, 100)

但如果只想改動一隻狗的 num_legs 數量呢? 修改 instance 的 class variable 就好

In [56]:
jack.num_legs = 4
tom.num_legs = 5

jack.num_legs, tom.num_legs

(4, 5)

上面的做法建立了一個 instance variable (由 class variable shallow copy 而來)，並且名稱與 class variable 是相同的  
以後在呼叫這個變數時，改為存取 instance variable  
+ 此行為**不會**影響到所有 instance 的 class variable

In [57]:
jack.num_legs, jack.__class__.num_legs

(4, 100)

In [58]:
Dog.num_legs

100

## A Dog-free Example
以下內容我們不去傷害狗狗了，改為介紹在現實有用的例子  
    
每當建立新的 CountedObject instance，num_instances 變數就會+1

In [59]:
class CountedObject:
  num_instances = 0
  def __init__(self):
    self.__class__.num_instances += 1

In [60]:
CountedObject.num_instances

0

In [61]:
CountedObject().num_instances

1

In [62]:
CountedObject().num_instances

2

以下的範例是錯誤的例子，每建立一次實體， `self.num_instances` 就+1，  
改動到的只是 instance 所建立的 shallow copy 而已

In [63]:
# WARNING: This implementation contains a bug
class BuggyCountedObject:
  num_instances = 0
  def __init__(self):
    self.num_instances += 1 # without __class__

In [64]:
BuggyCountedObject.num_instances

0

In [65]:
BuggyCountedObject().num_instances

1

In [66]:
BuggyCountedObject().num_instances

1

**Key Takeaways**
+ Class variables are for data shared by all instances of a class.
They belong to a class, not a specific instance and are shared among all instances of a class.
+ Instance variables are for data that is unique to each instance.
They belong to individual object instances and are not shared
among the other instances of a class. Each instance variable
gets a unique backing store specific to the instance.
+ Because class variables can be “shadowed” by instance variables of the same name, it’s easy to (accidentally) override
class variables in a way that introduces bugs and odd behavior.

# 4.8 Instance, Class, and Static Methods Demystified
In this chapter you’ll see what’s behind class methods, static methods,
and regular instance methods in Python.
  
**Instance Methods** 
+ 最簡單且常用，self 指向 instance，可用來存取內部成員，也能透過 `self.__class__` 來修改 class

**Class Methods**
+ cls 參數指向 class(而不是 instance)，不能用來更改 object instance ，但能更改 class attribute

**Static Methods**
+ 沒有 cls 和 self，無法存取內部成員，只能用來處理外部輸入資料


In [67]:
class MyClass:

  def method(self):
    return 'instance method called', self

  @classmethod
  def classmethod(cls):
    return 'class method called', cls

  @staticmethod
  def staticmethod():
    return 'static method called'

從 instance 層面呼叫

In [68]:
obj = MyClass()
# 回傳 instance 實體位址
obj.method() 

('instance method called', <__main__.MyClass at 0x7fe2d3c79c90>)

In [69]:
# 這行做的事跟上面一樣，把 instance 本身當作第一個參數
MyClass.method(obj)

('instance method called', <__main__.MyClass at 0x7fe2d3c79c90>)

In [70]:
obj.classmethod() # 回傳 class 本身(class 也是一種 object)

('class method called', __main__.MyClass)

從 class 層面呼叫

In [71]:
MyClass.classmethod()

('class method called', __main__.MyClass)

In [72]:
MyClass.staticmethod()

'static method called'

In [73]:
# 第一個參數需要是 instance，因此不能執行
# MyClass.method() 

## Delicious Pizza Factories With @classmethod
以下將介紹更實用的情況，讓大家了解如何選擇 method 種類

In [74]:
class Pizza:
  def __init__(self, ingredients):
    self.ingredients = ingredients
  def __repr__(self):
    return f'Pizza({self.ingredients!r})'

Pizza(['cheese', 'tomatoes'])

Pizza(['cheese', 'tomatoes'])

我們都知道隨著pizza配料不同，就會變成不同pizza  
如果想要快速創建出不同口味的 pizza 呢?
可以用 classmethod 作為 **factory
functions** 來快速創建出不同口味的實體

> from [wiki](https://en.wikipedia.org/wiki/Factory_(object-oriented_programming)):  
In object-oriented programming (OOP), a factory is an object for creating other objects 

In [75]:
class Pizza:
  def __init__(self, ingredients):
    self.ingredients = ingredients
  
  def __repr__(self):
    return f'Pizza({self.ingredients!r})'
  
  @classmethod
  def margherita(cls):
    return cls(['mozzarella', 'tomatoes'])
  
  @classmethod
  def prosciutto(cls):
    return cls(['mozzarella', 'tomatoes', 'ham'])

In [76]:
Pizza.margherita()

Pizza(['mozzarella', 'tomatoes'])

## When To Use Static Methods
在呼叫 area() 時，透過 static method circle_area 來計算  
circle_area 不能存取 cls 或是 self
+ 明確表示這個 method 是可獨立運作的
+ 避免修改到內部參數
+ 方便撰寫測試程式

In [77]:
import math

class Pizza:
  def __init__(self, radius, ingredients):
    self.radius = radius
    self.ingredients = ingredients
  
  def __repr__(self):
    return (f'Pizza({self.radius!r}, '
        f'{self.ingredients!r})')
  
  def area(self):
    return self.circle_area(self.radius)

  @staticmethod
  def circle_area(r):
    return r ** 2 * math.pi

**Key Takeaways**
+ Instance methods need a class instance and can access the instance through self.
+ Class methods don’t need a class instance. They can’t access the
instance (self) but they have access to the class itself via cls.
+ Static methods don’t have access to cls or self. They work like
regular functions but belong to the class’ namespace.
+ Static and class methods communicate and (to a certain degree)
enforce developer intent about class design. This can have definite maintenance benefits.